In [7]:
import tkinter as tk
from tkinter import ttk
from aslm.view.custom_widgets.validation import ValidatedMixin
from decimal import Decimal, InvalidOperation

class ValidatedEntry(ValidatedMixin, ttk.Entry):
    def __init__(self, *args, precision=0, required=False, min_var=None, max_var=None, focus_update_var=None, min='-Infinity', max='Infinity', **kwargs):
        super().__init__(*args, **kwargs)
        self.resolution = Decimal(precision) # Number for precision given on creation
        self.precision = (self.resolution.normalize().as_tuple().exponent) # Precision of number as exponent
        self.variable = kwargs.get('textvariable') or tk.StringVar
        self.min = min
        self.max = max
        self.required = required
        self.undo_history = []
        self.redo_history = []
        
    # Dynamic range checker
        # if min_var:
        #     self.min_var = min_var
        #     self.min_var.trace_add('w', self._set_minimum)
        # if max_var:
        #     self.max_var = max_var
        #     self.max_var.trace_add('w', self._set_maximum)
        # self.focus_update_var = focus_update_var
        # self.bind('<FocusOut>', self._set_focus_update_var)

    # Binding key press
        # self.bind('<Key>', self.key_print)

    # Update history
        self.bind("<FocusOut>", self.add_history)

    # Undo/Redo
        self.bind("z", self.undo)
        self.bind("y", self.redo)

    
    # All widgets that want to use undo redo will need this function and the undo_history, redo_history lists
    def add_history(self, event):
        if event.type == '10':
            if self.get() != '':
                if not self.undo_history or self.get() != self.undo_history[-1]:
                    self.undo_history.append(self.get())
                    print(event.widget.undo_history)
                    print("widget is accessed : ",event.widget.redo_history)

    def undo(self, event):
        if event.state == 4:
            if len(self.undo_history) > 1:
                self.delete(0, tk.END)
                self.insert(tk.END, self.undo_history[-2])
                self.redo_history.append(self.undo_history.pop())
            elif len(self.undo_history) == 1:
                self.delete(0, tk.END)
                self.redo_history.append(self.undo_history.pop())
            
    
    def redo(self, event):
        if event.state == 4:
            if self.redo_history:
                self.delete(0, tk.END)
                self.insert(tk.END, self.redo_history[-1])
                self.undo_history.append(self.redo_history.pop())

    def key_print(self, event):
        print(event.state)
        print(event.keysym)
        print(event.keycode)
        print(event)


In [8]:
root = tk.Tk()
entry = ValidatedEntry(root, takefocus=False)
entry1 = ttk.Entry(root)
entry.pack()
entry1.pack()
root.mainloop()

['hello']
widget is accessed :  []
['hello', 'there']
widget is accessed :  []
['hello', 'there', 'again']
widget is accessed :  []
